In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.utils import shuffle # library to shuffle the dataframes
import seaborn as sns # library to plot and visualize data
from sklearn.model_selection import train_test_split # library to split data into trining and testing set

In [4]:
# Defining the datatype of each column for reading the CSV file
dataTypes = {
    "UniqueID": np.uint16,
    "disbursed_amount": np.float64,
    "asset_cost": np.float64,
    "ltv": np.float64,
    "branch_id": np.object,
    "supplier_id": np.object,
    "manufacturer_id": np.object,
    "Current_pincode_ID": np.object,
    "Date.of.Birth": np.object,
    "Employment.Type": np.object,
    "DisbursalDate": np.object,
    "State_ID": np.object,
    "Employee_code_ID": np.object,
    "MobileNo_Avl_Flag": np.uint8,
    "Aadhar_flag": np.uint8,
    "PAN_flag": np.uint8,
    "VoterID_flag": np.uint8,
    "Driving_flag": np.uint8,
    "Passport_flag": np.uint8,
    "PERFORM_CNS.SCORE": np.uint16,
    "PERFORM_CNS.SCORE.DESCRIPTION": np.object,
    "PRI.NO.OF.ACCTS": np.uint8,
    "PRI.ACTIVE.ACCTS": np.uint8,
    "PRI.OVERDUE.ACCTS": np.uint8,
    "PRI.CURRENT.BALANCE": np.float64,
    "PRI.SANCTIONED.AMOUNT": np.float64,
    "PRI.DISBURSED.AMOUNT": np.float64,
    "SEC.NO.OF.ACCTS": np.uint8,
    "SEC.ACTIVE.ACCTS": np.uint8,
    "SEC.OVERDUE.ACCTS": np.uint8,
    "SEC.CURRENT.BALANCE": np.float64,
    "SEC.SANCTIONED.AMOUNT": np.float64,
    "SEC.DISBURSED.AMOUNT": np.float64,
    "PRIMARY.INSTAL.AMT": np.float64,
    "SEC.INSTAL.AMT": np.float64,
    "NEW.ACCTS.IN.LAST.SIX.MONTHS": np.uint8,
    "DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS": np.uint8,
    "AVERAGE.ACCT.AGE": np.object,
    "CREDIT.HISTORY.LENGTH": np.object,
    "NO.OF_INQUIRIES": np.uint8,
    "loan_default": np.uint8
}

# Input data files are available in the "../data/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
trainRaw = pd.read_csv("data/train.csv", dtype=dataTypes)

In [32]:
columnsToDrop = {
    "UniqueID",
    "Date.of.Birth",
    "DisbursalDate",
    "PERFORM_CNS.SCORE",
    "PERFORM_CNS.SCORE.DESCRIPTION",
    'Employment.Type',
    #"supplier_id",
    #'branch_id',
    #'manufacturer_id',
    #'Current_pincode_ID',
    #'State_ID',
    #'Employee_code_ID',
    'MobileNo_Avl_Flag',
    "PAN_flag",
    "Driving_flag",
    'Passport_flag',
    'SEC.OVERDUE.ACCTS',
    'SEC.ACTIVE.ACCTS',
    'SEC.INSTAL.AMT',
    "SEC.INSTAL.AMT",
    "SEC.CURRENT.BALANCE",
    'SEC.SANCTIONED.AMOUNT',
    'SEC.DISBURSED.AMOUNT',
    'SEC.NO.OF.ACCTS'
}

def parseDuration(x):
    x_yrs = x.split()[0]
    x_mon = x.split()[1]
    n_years = x_yrs[:(len(x_yrs)-3)]
    n_mon = x_mon[:(len(x_mon)-3)]
    return int(n_years) * 12 + int(n_mon)

def parseDate(v):
    if v.split("-")[-1]=='00' or v.split("-")[-1]=='18':
        return "-".join(v.split("-")[:-1])+'-20'+"".join(v.split("-")[-1])
    else:
        return "-".join(v.split("-")[:-1])+'-19'+"".join(v.split("-")[-1])
    
def getScoreGrade(desc):
    grade = desc.split("-")
    if len(grade) != 1:
        return grade[0]
    else:
        return 'Z'

#Pre-Process the data
def removeOutliers(data):
    data['disbursed_amount'] = np.clip(data['disbursed_amount'], a_min=25000, a_max = 80000)
    data['asset_cost'] = np.clip(data['asset_cost'], a_min=45000, a_max = 100000)
    data['ltv'] = np.clip(data['ltv'], a_min=40, a_max = 100)
    data['PRI.NO.OF.ACCTS'] = np.clip(data['PRI.NO.OF.ACCTS'], a_min=0, a_max = 10)
    data['PRI.ACTIVE.ACCTS'] = np.clip(data['PRI.ACTIVE.ACCTS'], a_min=0, a_max = 10)
    data['PRI.OVERDUE.ACCTS'] = np.clip(data['PRI.OVERDUE.ACCTS'], a_min=0, a_max = 5)
    data['PRI.CURRENT.BALANCE'] = np.clip(data['PRI.CURRENT.BALANCE'], a_min=0, a_max = 500000)
    data['PRI.SANCTIONED.AMOUNT'] = np.clip(data['PRI.SANCTIONED.AMOUNT'], a_min=0, a_max = 500000)
    data['PRI.DISBURSED.AMOUNT'] = np.clip(data['PRI.DISBURSED.AMOUNT'], a_min=0, a_max = 500000)
    data['SEC.NO.OF.ACCTS'] = np.clip(data['SEC.NO.OF.ACCTS'], a_min=0, a_max = 5)
    data['SEC.ACTIVE.ACCTS'] = np.clip(data['SEC.ACTIVE.ACCTS'], a_min=0, a_max = 3)
    data['SEC.OVERDUE.ACCTS'] = np.clip(data['SEC.OVERDUE.ACCTS'], a_min=0, a_max = 1)
    data['SEC.CURRENT.BALANCE'] = np.clip(data['SEC.CURRENT.BALANCE'], a_min=0, a_max = 100000)
    data['SEC.SANCTIONED.AMOUNT'] = np.clip(data['SEC.SANCTIONED.AMOUNT'], a_min=0, a_max = 100000)
    data['SEC.DISBURSED.AMOUNT'] = np.clip(data['SEC.DISBURSED.AMOUNT'], a_min=0, a_max = 100000)
    data['PRIMARY.INSTAL.AMT'] = np.clip(data['PRIMARY.INSTAL.AMT'], a_min=0, a_max = 10000)
    data['SEC.INSTAL.AMT'] = np.clip(data['SEC.INSTAL.AMT'], a_min=0, a_max = 2000)
    data['NEW.ACCTS.IN.LAST.SIX.MONTHS'] = np.clip(data['NEW.ACCTS.IN.LAST.SIX.MONTHS'], a_min=0, a_max = 4)
    data['DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS'] = np.clip(data['DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS'], a_min=0, a_max = 5)
    data['AVERAGE.ACCT.AGE'] = np.clip(data['AVERAGE.ACCT.AGE'], a_min=0, a_max = 48)
    data['CREDIT.HISTORY.LENGTH'] = np.clip(data['CREDIT.HISTORY.LENGTH'], a_min=0, a_max = 120)
    data['NO.OF_INQUIRIES'] = np.clip(data['NO.OF_INQUIRIES'], a_min=0, a_max = 20)

#Pre-Process the data
def preProcessData(raw):
    data = raw.copy()
    
    data['Date.of.Birth'] = pd.to_datetime(data['Date.of.Birth'].apply(parseDate), format='%d-%m-%Y', errors='coerce')
    data['DisbursalDate'] = pd.to_datetime(data['DisbursalDate'].apply(parseDate), format='%d-%m-%Y')
    data['AgeAtDisbursal'] = (data['DisbursalDate'] - data['Date.of.Birth']).astype('<m8[Y]')
    
    data['Employment.Type'].fillna('Other', inplace=True)
    
    data['AVERAGE.ACCT.AGE'] = data['AVERAGE.ACCT.AGE'].apply(parseDuration)
    data['CREDIT.HISTORY.LENGTH'] = data['CREDIT.HISTORY.LENGTH'].apply(parseDuration)
    
    removeOutliers(data)
    
    data['Sanctioned'] = (data['asset_cost'] * (data['ltv'] / 100)).astype(np.float64)
    data['TOTAL.ACTIVE.ACCTS'] = (data['PRI.ACTIVE.ACCTS'] + data['SEC.ACTIVE.ACCTS']).astype(np.int64)
    data['TOTAL.CURRENT.BALANCE'] = (data['PRI.CURRENT.BALANCE'] + data['SEC.CURRENT.BALANCE']).astype(np.float64)
    data['TOTAL.DISBURSED.AMOUNT'] = (data['PRI.DISBURSED.AMOUNT'] + data['SEC.DISBURSED.AMOUNT']).astype(np.int64)
    data['TOTAL.NO.OF.ACCTS'] = (data['PRI.NO.OF.ACCTS'] + data['SEC.NO.OF.ACCTS']).astype(np.int64)
    data['TOTAL.OVERDUE.ACCTS'] = (data['PRI.OVERDUE.ACCTS'] + data['SEC.OVERDUE.ACCTS']).astype(np.int64)
    data['TOTAL.CLEAN.ACCTS'] = (data['TOTAL.NO.OF.ACCTS'] - data['TOTAL.OVERDUE.ACCTS']).astype(np.int64)
    data['NO.OF.ACC.BF.SIX.MONTH'] = (data['TOTAL.NO.OF.ACCTS'] - data['NEW.ACCTS.IN.LAST.SIX.MONTHS']).astype(np.int64)
    data['OVERDUE.ACC.BF.SIX.MONTHS'] = (data['TOTAL.OVERDUE.ACCTS'] - data['DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS']).astype(np.int64)
    data['TOTAL.DEACTIVE.ACCTS'] = (data['TOTAL.NO.OF.ACCTS'] - data['TOTAL.ACTIVE.ACCTS']).astype(np.int64)
    data['TOTAL.INSTL.AMOUNT'] = (data['PRIMARY.INSTAL.AMT'] + data['SEC.INSTAL.AMT']).astype(np.float64)
    data['TOTAL.CLEARED.ACCTS']= (data['TOTAL.NO.OF.ACCTS'] - data['TOTAL.ACTIVE.ACCTS'] + data['TOTAL.OVERDUE.ACCTS']).astype(np.int64)

    data['PERFORM_CNS.SCORE.DESCRIPTION'].replace({
        'A-Very Low Risk':'Medium Risk',
        'B-Very Low Risk':'Low Risk',
        'C-Very Low Risk':'Medium Risk',
        'D-Very Low Risk':'Low Risk',
        'E-Low Risk':'Medium Risk',
        'F-Low Risk':'Medium Risk',
        'G-Low Risk':'Medium Risk',
        'H-Medium Risk':'High Risk',
        'I-Medium Risk':'High Risk',
        'J-High Risk':'High Risk',
        'K-High Risk':'Very HRisk',
        'L-Very High Risk':'Very HRisk',
        'M-Very High Risk':'Very HRisk',
        'Not Scored: More than 50 active Accounts found':'Low Risk',
        'No Bureau History Available':'Medium Risk', 
        'Not Scored: Only a Guarantor':'High Risk',
        'Not Scored: Only a Guarantor':'High Risk',        
        'Not Scored: No Activity seen on the customer (Inactive)':'Medium Risk',
        'Not Scored: No Updates available in last 36 months':'Medium Risk',
        'Not Scored: Not Enough Info available on the customer':'Medium Risk',        
        'Not Scored: Sufficient History Not Available':'Very HRisk'}, inplace=True)
    
    data['Low_Risk'] = np.where(data['PERFORM_CNS.SCORE.DESCRIPTION'].str.contains('Low Risk'), 1, 0)
    data['Medium_Risk'] = np.where(data['PERFORM_CNS.SCORE.DESCRIPTION'].str.contains('Medium Risk'), 1, 0)
    data['High_Risk'] = np.where(data['PERFORM_CNS.SCORE.DESCRIPTION'].str.contains('High Risk'), 1, 0)
    data['Very_High_Risk'] = np.where(data['PERFORM_CNS.SCORE.DESCRIPTION'].str.contains('Very HRisk'), 1, 0)
    
    X = pd.get_dummies(data["Employment.Type"], prefix="Employment.Type", drop_first=False, dtype=np.uint8)
    data = pd.concat([data, X], axis=1)

    index = 0
    prev = 0
    for i in range(100, 1001, 100):
        data.loc[(data['PERFORM_CNS.SCORE'] < i) & (data['PERFORM_CNS.SCORE'] >= prev), 'PERFORM_CNS.SCORE'] = index
        index = index + 1
        prev = i
        
    X = pd.get_dummies(data["PERFORM_CNS.SCORE"], prefix="PERFORM_CNS.SCORE", drop_first=False, dtype=np.uint8)
    data = pd.concat([data, X], axis=1)
    
    data = data.drop(columnsToDrop, axis=1)
    
    return data

In [33]:
train = preProcessData(trainRaw)

In [34]:
X = train.drop('loan_default', 1)
y = train['loan_default']

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=41)

In [36]:
X_train.to_csv("data\X_train.csv", sep=',', encoding='utf-8', index=False, header=True)
X_test.to_csv("data\X_test.csv", sep=',', encoding='utf-8', index=False, header=True)
y_train.to_csv("data\y_train.csv", sep=',', encoding='utf-8', index=False, header=True)
y_test.to_csv("data\y_test.csv", sep=',', encoding='utf-8', index=False, header=True)